# CC3001 Otoño 2021 Tarea 4

### Profesores
Sección 1 Nelson Baloian/Patricio Poblete •
Sección 2 Benjamín Bustos •
Sección 3 Sebastián Ferrada


# Ordenamiento Topológico

El problema de *ordenamiento topológico* consiste en encontrar un orden total que sea compatible con un orden parcial dado.

Para formular el problema de manera más concreta, lo describiremos en términos de cursos y sus requisitos.

Consideremos un conjunto de cursos, para cada uno de los cuales se tiene la lista de sus requisitos. Para simplificar, supondremos que no existen requisitos simultáneos ni de tipo "o", sólo de tipo "y".

Como ejemplo, tomemos un conjunto de cursos numerados $0,1,\ldots$ para cada uno de los cuales se indica su lista de requisitos:

<pre>
0:
1: 0
2: 1 4
3:
4: 3
5:
6: 0 3 5
7: 1 4 6
8:
9: 8
10: 3 9
</pre>

Observe en particular que los cursos 0, 3, 5 y 8 no tienen requisitos.

Esto se puede visualizar en la siguiente malla:

![malla](https://dcc.uchile.cl/ppoblete/cc3001/2023-2/malla.png)

Lo que se pide es generar un orden en que se podría ir aprobando los cursos, de tal manera que al tomar cada curso, sus requisitos ya se encuentren cumplidos. Por ejemplo, un orden factible (entre muchos otros) podría ser:

<pre>
3 0 4 1 8 2 5 9 6 10 7
</pre>

Usted puede suponer que la malla no contiene ciclos, por lo cual el problema siempre tiene solución.

Para lograr lo pedido, se propone usar el siguiente algoritmo:

* Comenzar con una lista vacía
* Encontrar un curso con cero flechas entrantes (esto es, un curso sin requisitos) y agregar su número al final de la lista. Borrar ese curso de la malla, junto con todas sus flechas salientes.
* Repetir el punto anterior hasta que todos los cursos hayan sido agregados a la lista y no quede ningún curso en la malla.






# Implementación eficiente del algoritmo

Para implementar este algoritmo, definiremos una clase ``Malla`` con los siguientes métodos:

* un constructor que recibe un parámetro ``n`` (el número de cursos) y una lista (de Python) ``L`` que contiene una lista de pares ``[i,j]``, que corresponden a todas las flechas desde un curso ``i`` a un curso ``j`` en la malla. Con esta información, el constructor debe crear una representación interna de la malla, como se describe más adelante.

* ``m.extrae_curso()`` que encuentra un curso que no tenga requisitos en la malla (cero flechas entrantes), elimina ese curso de la malla (incluyendo eliminar todas las flechas salientes de él) y retorna el número del curso encontrado

* ``m.esta_vacia()`` que retorna ``True`` cuando la malla está vacía

Utilizando esta clase, es muy directo escribir un programa que implemente el algoritmo, utilizando el primer método para construir la malla y los dos restantes para generar el ordenamiento topológico.

Por supuesto, la eficiencia de la implementación dependerá de cómo funcione internamente la clase. Para esto, se deberán manejar las estructuras que se describen a continuación.

## Arreglo de flechas hacia adelante (``post``)

La lista de cursos con sus requisitos (a veces llamados "pre-requisitos") indicada anteriormente se puede interpretar como una lista de todos los cursos que apuntan en la malla hacia un curso dado. Acá utilizaremos lo opuesto: una lista de los "post-requisitos" de un curso dado, esto es, de los cursos que se pueden tomar una vez que uno ha aprobado dicho curso.

Este será un arreglo ``post`` en donde ``post[i]`` será un puntero a una lista enlazada de todos los cursos hacia los cuales apunta con una flecha el curso ``i`` en la malla.

Ejemplo:

![post](https://dcc.uchile.cl/ppoblete/cc3001/2023-2/post.png)

## Arreglo del número de flechas entrantes para cada curso (``num_pre``)

Este será un arreglo en donde el casillero ``num_pre[j]`` almacena el número de flechas entrantes al curso ``j`` en la malla (esto es, el número de pre-requisitos de ese curso).

Ejemplo:

![numpre](https://dcc.uchile.cl/ppoblete/cc3001/2023-2/numpre.png)

## Lista de los cursos con cero flechas entrantes (``lista_cero``)

Esta es una lista enlazada de todos los cursos que tienen cero flechas entrantes y que todavía no han sido eliminados de la malla.

Ejemplo:

![listacero](https://dcc.uchile.cl/ppoblete/cc3001/2023-2/listacero.png)

## Implementación de los métodos

El constructor debe inicialmente generar una tabla ``post`` de tamaño ``n`` con todas sus listas vacías, y una tabla ``num_pre`` en que los casilleros de todos los cursos (del 0 al ``n-1``) tienen valor cero (porque no tienen todavía flechas entrantes).

A continuación el constructor debe procesar la lista ``L``, agregando el nodo ``j`` a la lista ``post[i]`` y sumándole 1 al contador ``num_pre[j]``.

Una vez concluido este proceso, debe recorrer la tabla ``num_pre`` para formar la lista ``lista_cero`` de todos los cursos que tienen cero flechas entrantes.

El método ``extrae_curso()`` debe extraer un curso de la lista ``lista_cero`` y luego recorrer la lista ``post`` de ese curso, restándole 1 al contador de flechas entrantes de cada curso que esté en esa lista. Si con eso el curso queda con cero flechas entrantes, debe agregarse a la lista ``lista_cero``.

Escriba a continuación la definición de la clase ``Malla``, modificando el código que se le entrega:


In [1]:
import numpy as np

class Nodo:
  def __init__(self, info, sgte=None):
    self.info=info
    self.sgte=sgte

class Lista:
    def __init__(self):
        self.primero=None
        
    def insertar_al_inicio(self,info):
        self.primero=Nodo(info,self.primero)


    def imprimir(self):
        p=self.primero
        while p is not None:
            print(p.info, end=" ")
            p=p.sgte
        print()
    

class Malla:
  def __init__(self,n,L):
      self.n = n
      self.L = L
      self.post = [Lista() for _ in range(0, n)]
      self.num_pre = np.zeros(n)
      
    
      for l in self.L:
        self.post[l[0]].insertar_al_inicio(l[1])
        self.num_pre[l[1]] += 1

      # print("num_pre:", self.num_pre)

      self.lista_cero = []
      for k in range(0,len(self.num_pre)):
         if self.num_pre[k] == 0:
            self.lista_cero.append(k)

      # print("lista_cero:", self.lista_cero)



  def extrae_curso(self):
      curso = self.lista_cero[0]
      this_post = self.post[curso]
      info = this_post.primero
      while info is not None:
         if self.num_pre[info.info] > 0:
            self.num_pre[info.info] -= 1
            if self.num_pre[info.info] == 0:
              self.lista_cero.append(info.info)
            info = info.sgte
         
      # print("nueva lista cero", self.lista_cero)
      return self.lista_cero.pop(0)
      

  def esta_vacia(self):
    if self.lista_cero == []:
      return True
    else:
      return False

---
# Juntando todo lo anterior:

Usted debe escribir una función ``ordena_topologico(n,L)`` donde ``n`` es el número de cursos y ``L`` es una lista (de Python) que contien todos los pares ``[i,j]`` para ser entregados al constructor de la malla.
La función debe retornar una lista (de Python) con la secuencia pedida de cursos en un orden que sea compatible con los requisitos de la malla.

In [2]:
def ordena_topologico(n,L):
  m = Malla(n,L)
  malla_ordenada = []
  while not m.esta_vacia():
    malla_ordenada.append(m.extrae_curso())
  return malla_ordenada
  

# Probando el programa

En primer lugar vamos a definir una función para chequear que una solución es correcta:

In [3]:
def check(n,L,orden):
  if len(orden)!=n:
    print("ERROR")
    return
  pos=np.full(n,-1)
  for k in range(n):
    if pos[orden[k]]!=-1: # número repetido en la solución
      print("ERROR")
      return
    pos[orden[k]]=k
  for flecha in L:
    (i,j)=(flecha[0],flecha[1])
    if pos[i]>pos[j]:
      print("ERROR")
      return
  print("OK")


A continuación debe ejecutar los siguientes trozos de código para probar su programa:

In [4]:
# Probamos primero con una malla pequeña
n=5
L=[[3,1],[4,1],[3,2],[4,2],[1,0],[2,0]]

print("Primero probamos con una solución errónea, debe decir ERROR:")
orden=[1,3,2,4,0]
print(orden)
check(n,L,orden)

print("Ahora probamos con su solución, debe decir OK:")
orden=ordena_topologico(n,L)
print("orden:", orden)
check(n,L,orden)

Primero probamos con una solución errónea, debe decir ERROR:
[1, 3, 2, 4, 0]
ERROR
Ahora probamos con su solución, debe decir OK:
orden: [3, 4, 2, 1, 0]
OK


In [5]:
print("Finalmente probamos su programa con el ejemplo del enunciado, debe decir OK")
n=11
L=[[0,1],[0,6],
   [1,2],[1,7],
   [3,4],[3,6],[3,10],
   [4,2],[4,7],
   [5,6],[6,7],
   [8,9],
   [9,10]]


orden=ordena_topologico(n,L)
print(orden)
check(n,L,orden)

Finalmente probamos su programa con el ejemplo del enunciado, debe decir OK
[0, 3, 5, 8, 1, 4, 6, 9, 2, 7, 10]
OK


## ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Documentar adecuadamente su código. Cite todas las fuentes de información utilizadas. **No olvide poner su nombre en el encabezamiento**.